In [16]:
# remove columns; careful when running, will remove columns from prediction dataset
pattern = "-0$"

# test_causenet_predictions = pd.read_pickle(
#     os.path.join(constants.CEPH_PATH, "test_causenet_predictions.pkl")
# )
# print(list(test_causenet_predictions.filter(regex=pattern)))
# columns = list(test_causenet_predictions.filter(regex=pattern))
# test_causenet_predictions = test_causenet_predictions.drop(columns, axis=1)
# test_causenet_predictions.to_pickle(
#     os.path.join(constants.CEPH_PATH, "test_causenet_predictions.pkl")
# )
# print(list(test_causenet_predictions.filter(regex=pattern)))

# sentence_test_causenet_predictions = pd.read_pickle(
#     os.path.join(constants.CEPH_PATH, "sentence_test_causenet_predictions.pkl")
# )
# columns = list(sentence_test_causenet_predictions.filter(regex=pattern))
# sentence_test_causenet_predictions = sentence_test_causenet_predictions.drop(columns, axis=1)
# sentence_test_causenet_predictions.to_pickle(
#     os.path.join(constants.CEPH_PATH, "sentence_test_causenet_predictions.pkl")
# )
# print(list(sentence_test_causenet_predictions.filter(regex=pattern)))

['medical_score-cause-term_domain_specificity-pubmed-(1, 1)-0', 'medical_score-cause-term_domain_specificity-textbook-(1, 1)-0', 'medical_score-cause-term_domain_specificity-pubmed_central-(1, 1)-0', 'medical_score-cause-term_domain_specificity-encyclopedia-(1, 1)-0', 'medical_score-cause-term_domain_specificity-pubmed-(1, 2)-0', 'medical_score-cause-term_domain_specificity-textbook-(1, 2)-0', 'medical_score-cause-term_domain_specificity-pubmed_central-(1, 2)-0', 'medical_score-cause-term_domain_specificity-encyclopedia-(1, 2)-0', 'medical_score-cause-term_domain_specificity-pubmed-(1, 3)-0', 'medical_score-cause-term_domain_specificity-textbook-(1, 3)-0', 'medical_score-cause-term_domain_specificity-pubmed_central-(1, 3)-0', 'medical_score-cause-term_domain_specificity-encyclopedia-(1, 3)-0', 'medical_score-cause-contrastive_weight-pubmed-(1, 1)-0', 'medical_score-cause-contrastive_weight-textbook-(1, 1)-0', 'medical_score-cause-contrastive_weight-pubmed_central-(1, 1)-0', 'medical_sc

In [2]:
import sys
import os

parent_path = ".."
sys.path.append(os.path.abspath(parent_path))

In [3]:
import nltk

nltk.download("stopwords")

import pickle5
from importlib import reload
import pandas as pd
import numpy as np
import health_causenet
from health_causenet import constants
from health_causenet.causenet import (
    CauseNet,
    contrastive_weight,
    term_domain_specificity,
    discriminative_weight,
)
import quickumls
from tqdm.autonotebook import tqdm
import os
import pathlib
import json
from IPython.display import clear_output
from health_bert import health_bert

tqdm.pandas()

tokenizer = nltk.tokenize.TreebankWordTokenizer()

[nltk_data] Downloading package stopwords to /mnt/ceph/storage/data-
[nltk_data]     tmp/current//fschlatt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2022-08-30 12:16:54.236695: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-08-30 12:17:36.806356: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-08-30 12:17:36.814721: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-30 12:17:36.814749: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: gammaweb06
2022-08-30 12:17:36.814755: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: gammaweb06
2022-08-30 12:17:36.814843: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not f

In [3]:
def pickle_5_to_4(path):
    with open(path, "rb") as fh:
        data = pickle5.load(fh)
    data.to_pickle(path)

In [15]:
pickle_5_to_4(os.path.join(constants.CEPH_PATH, "test_causenet_predictions.pkl"))

In [ ]:
# Create test causenet

full_causenet = pd.DataFrame()
paths = sorted(pathlib.Path(constants.CAUSENET_PARQUET_PATH).glob("causenet_*.parquet"))
for path in tqdm(paths):
    from_file = pd.read_parquet(
        path, columns=["cause", "effect", "support", "reference", "sentence"]
    )
    full_causenet = full_causenet.append(from_file)
full_causenet = full_causenet.reset_index(drop=True)
print("parsing domain...")
print("computing counts...")
causenet = full_causenet.groupby(["cause", "effect", "support"]).size()
causenet.name = "count"
causenet = causenet.reset_index()

print("sorting by support...")
test_causenet_support = causenet.sort_values(
    ["support", "cause", "effect"], ascending=False
).reset_index(drop=True)
test_causenet_support = test_causenet_support.iloc[:1000].copy()
test_causenet_support["dataset"] = "support"
test_causenet_random_high = (
    causenet.loc[causenet.support >= 2].sample(1000, random_state=42).copy()
)
test_causenet_random_high["dataset"] = "random_support"
test_causenet_random_low = (
    causenet.loc[causenet.support == 1].sample(1000, random_state=42).copy()
)
test_causenet_random_low["dataset"] = "random_full"
test_causenet = pd.concat(
    [test_causenet_support, test_causenet_random_high, test_causenet_random_low]
)
test_causenet = test_causenet.reset_index(drop=True)

print("adding wikidata...")
test_wikidata = (
    pd.read_csv(os.path.join(constants.WIKIDATA_PATH, "wikidata-test.csv"), index_col=0)
    .drop_duplicates()
    .dropna(subset=["cause", "effect"])
    .reset_index(drop=True)
)
test_wikidata["dataset"] = "wikidata"
test_causenet = test_causenet.append(
    test_wikidata.loc[
        :, ["cause", "effect", "dataset", "cause_origin", "effect_origin"]
    ]
).reset_index(drop=True)
test_causenet = test_causenet.drop_duplicates(["cause", "effect", "dataset"])

print("adding practitioner...")
test_practitioner = pd.read_csv(
    os.path.join(constants.CEPH_PATH, "practitioner.csv"), index_col=0
)
test_practitioner = test_practitioner.dropna(subset=["medical"])
test_practitioner["evaluation"] = test_practitioner.medical.str.contains("y")
test_practitioner["dataset"] = "practitioner_full"
test_causenet = test_causenet.append(
    test_practitioner.loc[:, ["cause", "effect", "dataset"]]
).reset_index(drop=True)
test_practitioner.loc[
    ~test_practitioner.comment.isna(), "dataset"
] = "practitioner_unsure"
test_practitioner.loc[test_practitioner.comment.isna(), "dataset"] = "practitioner_sure"
test_causenet = test_causenet.append(
    test_practitioner.loc[:, ["cause", "effect", "dataset"]]
).reset_index(drop=True)

# Load and label evaluations

ignore_origins = [
    #     "wd:Q39833",  # microorganism
    #     "wd:Q178694",  # heredity
    #     "wd:Q289472",  # biogenic substance
    #     "wd:Q796194",  # medical procedure
    #     "wd:Q2826767",  # disease causative agent
    #     "wd:Q2996394",  # biological process
    #     "wd:Q5850078",  # etiology
    #     "wd:Q7189713",  # physiological condition
    #     "wd:Q15788410",  # state of consciousness
    #     "wd:Q86746756",  # medicinal product
    #     "wd:Q87075524",  # health risk
]

with open(constants.MANUAL_EVALUATION_PATH, "r") as file:
    manual_eval_dict = json.load(file)
data = []
for key, value in manual_eval_dict.items():
    cause, effect = key.split("->")
    data.append({"cause": cause, "effect": effect, "evaluation": value})
test_causenet_manual = test_causenet.loc[
    test_causenet.dataset.isin(["support", "random_support", "random_full", "count"])
]
manual_eval = pd.DataFrame(data)
evaluation = manual_eval.set_index(["cause", "effect"]).reindex(
    test_causenet_manual.set_index(["cause", "effect"]).index
)

end = False
to_label = test_causenet_manual.loc[evaluation.isna().values].values
for idx, row in enumerate(to_label):
    cause = row[0]
    effect = row[1]
    while True:
        clear_output(wait=True)
        inp = input(
            f"{idx+1}/{len(to_label)} ({len(to_label) - idx}) [{cause}] -> [{effect}]"
        )
        if inp == "c":
            end = True
            break
        try:
            val = int(inp)
            if val in (0, 1):
                key = f"{cause}->{effect}"
                manual_eval_dict[key] = val
                break
        except:
            pass
        print(f"invalid input: {inp}, needs to be either 1 or 0")
    if end:
        break

with open(constants.MANUAL_EVALUATION_PATH, "w") as file:
    json.dump(manual_eval_dict, file, indent=4)

data = []
for key, value in manual_eval_dict.items():
    cause, effect = key.split("->")
    data.append({"cause": cause, "effect": effect, "evaluation": value})
manual_eval = pd.DataFrame(data)
evaluation = manual_eval.set_index(["cause", "effect"]).reindex(
    test_causenet_manual.set_index(["cause", "effect"]).index
)

print(f"eval missing for {evaluation.isna().sum().values[0]} relations")

evaluation = evaluation.evaluation

evaluation = evaluation.append(test_wikidata.set_index(["cause", "effect"]).evaluation)
evaluation = evaluation.append(
    test_practitioner.set_index(["cause", "effect"]).evaluation
)
evaluation = evaluation.append(
    test_practitioner.set_index(["cause", "effect"]).evaluation
)

test_causenet["evaluation"] = evaluation.values

print("parsing sentence test causenet")
sentences = (
    full_causenet.set_index(["cause", "effect", "support"])
    .loc[
        test_causenet.loc[
            ~test_causenet.support.isna(), ["cause", "effect", "support"]
        ].values.tolist(),
        "sentence",
    ]
    .reset_index()
    .drop_duplicates(["cause", "effect", "sentence"])
)
sentence_test_causenet = test_causenet.merge(
    sentences, on=["cause", "effect", "support"]
)
sentence_test_causenet = sentence_test_causenet.drop_duplicates(["dataset", "sentence"])
sentence_test_causenet = sentence_test_causenet.drop_duplicates(
    ["cause", "effect", "dataset"]
)
sentence_test_causenet_evaluation = pd.read_csv(
    os.path.join(constants.CEPH_PATH, "sentence_test_causenet_evaluations.csv"),
    index_col=0,
).rename({"label": "manual_evaluation"}, axis=1)
sentence_test_causenet = sentence_test_causenet.merge(
    sentence_test_causenet_evaluation, on=["cause", "effect", "sentence"], how="left"
)

del causenet
del full_causenet

# print("creating ctakes and metamap data")
# relations = pd.Series(
#     pd.unique(test_causenet.loc[:, ["cause", "effect"]].values.ravel())
# )
# relations = relations.loc[~relations.isin(["", " "])]
# sentence_relations = pd.Series(
#     pd.unique(sentence_test_causenet.loc[:, ["sentence"]].values.ravel())
# )
# relations = pd.concat([relations, sentence_relations]).reset_index(drop=True)
# ctakes_path = pathlib.Path(constants.CEPH_PATH).joinpath("ctakes")
# ctakes_path.mkdir(exist_ok=True)
# metamap_path = pathlib.Path(constants.CEPH_PATH).joinpath("metamap")
# metamap_path.mkdir(exist_ok=True)
# for idx, relation in enumerate(relations.values):
#     with ctakes_path.joinpath(f"relation_{idx + 1}.txt").open("w") as file:
#         file.write(relation)
# with metamap_path.joinpath("relations.txt").open("w") as file:
#     file.write("\n".join(f"{idx + 1}|{relation}" for idx, relation in enumerate(relations.values)))

test_causenet.to_pickle(os.path.join(constants.CEPH_PATH, "test_causenet.pkl"))
sentence_test_causenet.to_pickle(
    os.path.join(constants.CEPH_PATH, "sentence_test_causenet.pkl")
)
test_causenet

In [4]:
# load test_causenet
test_causenet = pd.read_pickle(os.path.join(constants.BASE_PATH, "test_causenet.pkl"))
sentence_test_causenet = pd.read_pickle(
    os.path.join(constants.BASE_PATH, "sentence_test_causenet.pkl")
)
test_causenet

,cause,effect,support,count,dataset,cause_origin,effect_origin,evaluation
0,accident,death,38.0,1458.0,support,NaN,NaN,0
1,pneumonia,death,37.0,1185.0,support,NaN,NaN,1
2,disease,death,37.0,1344.0,support,NaN,NaN,1
3,illness,death,36.0,571.0,support,NaN,NaN,1
4,heart attack,death,36.0,1005.0,support,NaN,NaN,1
...,...,...,...,...,...,...,...,...
14311,initial oxidation at carbon 5,thymine glycol,NaN,NaN,practitioner_unsure,NaN,NaN,1
14312,thyroid hormone deficits,language,NaN,NaN,practitioner_sure,NaN,NaN,0
14313,neuropeptide profile,decreased food intake,NaN,NaN,practitioner_sure,NaN,NaN,0
14314,disorder,white heads,NaN,NaN,practitioner_sure,NaN,NaN,0


In [5]:
# load cf and compute termhood scores
cf = pd.read_parquet(os.path.join(constants.CF_PATH, "cf.parquet"))
cf.loc[cf.num_terms == 1].sum().astype(str)
medical_termhood = {}
for corpus in list(cf.filter(regex=r".*_frequency_(?!open_domain)")):
    _cf = cf.loc[:, ["corpus_frequency_open_domain", corpus, "num_terms"]]
    corpus = corpus.replace("corpus_frequency_", "")
    medical_termhood[corpus] = {}
    print(f"{corpus}: computing term domain specificity...")
    medical_termhood[corpus]["term_domain_specificity"] = term_domain_specificity(
        _cf, np.e
    )
    print(f"{corpus}: computing contrastive weight...")
    medical_termhood[corpus]["contrastive_weight"] = contrastive_weight(_cf, np.e, 1)
    print(f"{corpus}: computing_discriminative weight...")
    medical_termhood[corpus]["discriminative_weight"] = (
        medical_termhood[corpus]["contrastive_weight"]
        * medical_termhood[corpus]["term_domain_specificity"]
    )
del cf

pubmed: computing term domain specificity...
pubmed: computing contrastive weight...
pubmed: computing_discriminative weight...
pubmed_central: computing term domain specificity...
pubmed_central: computing contrastive weight...
pubmed_central: computing_discriminative weight...
textbook: computing term domain specificity...
textbook: computing contrastive weight...
textbook: computing_discriminative weight...
encyclopedia: computing term domain specificity...
encyclopedia: computing contrastive weight...
encyclopedia: computing_discriminative weight...


In [15]:
# speed test
import time
import datetime

times = {
    "sentence": {
        "termhood": {1: [], 2: [], 3: []},
        "bert": {" ": []}, 
        "quickumls": {"full": [], "rx_sno": []}, 
        "scispacy": {"full": [], "rx_sno": []}, 
        "metamap": {"full": [160.638], "rx_sno": [120.277]},
        "ctakes": {"full": [0], "rx_sno": [212.711]},
    }, 
    "phrase": {
        "termhood": {1: [], 2: [], 3: []},
        "bert": {" ": []}, 
        "quickumls": {"full": [], "rx_sno": []}, 
        "scispacy": {"full": [], "rx_sno": []},
        "metamap": {"full": [70.639], "rx_sno": [49.636]},
        "ctakes": {"full": [0], "rx_sno": [119.684]},
    }
}

n = 10

model = health_bert.HealthBert.load_from_checkpoint(
    constants.BASE_PATH + "models/health_bert/" + "pubmedbert_pubmed_sentence.ckpt"
)

for _ in tqdm(range(n), total=n):
    # termhood
    for n_gram in (1, 2, 3):
        start = time.perf_counter()
        health_causenet.causenet._contrastive_score(
            test_causenet.loc[test_causenet.dataset == "random_full"],
            medical_termhood["encyclopedia"]["discriminative_weight"],
            p=1,
            n_gram_size=(1, n_gram),
            verbose=False,
        )
        elapsed = time.perf_counter() - start
        times["phrase"]["termhood"][n_gram].append(elapsed)
        start = time.perf_counter()
        health_causenet.causenet._contrastive_score(
            sentence_test_causenet.loc[sentence_test_causenet.dataset == "random_full"].assign(
                cause=sentence_test_causenet.sentence, effect=""
            ),
            medical_termhood["encyclopedia"]["discriminative_weight"],
            p=1,
            n_gram_size=(1, n_gram),
            verbose=False,
        )
        elapsed = time.perf_counter() - start
        times["sentence"]["termhood"][n_gram].append(elapsed)

    # bert
    start = time.perf_counter()
    health_causenet.causenet._health_bert(
        test_causenet.loc[test_causenet.dataset == "random_full"],
        model,
        verbose=False,
        batch_size=1,
    )
    elapsed = time.perf_counter() - start
    start = time.perf_counter()
    times["phrase"]["bert"][" "].append(elapsed)
    health_causenet.causenet._health_bert(
        sentence_test_causenet.loc[sentence_test_causenet.dataset == "random_full"].assign(
            cause=sentence_test_causenet.sentence, effect=""
        ),
        model,
        batch_size=1,
        verbose=False,
    )
    elapsed = time.perf_counter() - start
    times["sentence"]["bert"][" "].append(elapsed)

    # quickumls
    start = time.perf_counter()
    CauseNet.is_medical(
        test_causenet.loc[test_causenet.dataset == "random_full"],
        "quickumls",
        jaccard_threshold=0.9,
        umls_subset="full",
        st21pv=False,
        verbose=False,
    )
    elapsed = time.perf_counter() - start
    times["phrase"]["quickumls"]["full"].append(elapsed)
    start = time.perf_counter()
    CauseNet.is_medical(
        test_causenet.loc[test_causenet.dataset == "random_full"],
        "quickumls",
        jaccard_threshold=0.9,
        umls_subset="rx_sno",
        st21pv=False,
        verbose=False,
    )
    elapsed = time.perf_counter() - start
    times["phrase"]["quickumls"]["rx_sno"].append(elapsed)
    start = time.perf_counter()
    CauseNet.is_medical(
        sentence_test_causenet.loc[sentence_test_causenet.dataset == "random_full"].assign(
            cause=sentence_test_causenet.sentence, effect=""
        ),
        "quickumls",
        jaccard_threshold=0.9,
        umls_subset="full",
        st21pv=False,
        verbose=False,
    )
    elapsed = time.perf_counter() - start
    times["sentence"]["quickumls"]["full"].append(elapsed)
    start = time.perf_counter()
    CauseNet.is_medical(
        sentence_test_causenet.loc[sentence_test_causenet.dataset == "random_full"].assign(
            cause=sentence_test_causenet.sentence, effect=""
        ),
        "quickumls",
        jaccard_threshold=0.9,
        umls_subset="rx_sno",
        st21pv=False,
        verbose=False,
    )
    elapsed = time.perf_counter() - start
    times["sentence"]["quickumls"]["rx_sno"].append(elapsed)

    # scispacy
    start = time.perf_counter()
    CauseNet.is_medical(
        test_causenet.loc[test_causenet.dataset == "random_full"],
        "scispacy",
        threshold=0.9,
        umls_subset="full",
        model="en_core_sci_sm",
        st21pv=False,
        verbose=False,
    )
    elapsed = time.perf_counter() - start
    times["phrase"]["scispacy"]["full"].append(elapsed)
    start = time.perf_counter()
    CauseNet.is_medical(
        test_causenet.loc[test_causenet.dataset == "random_full"],
        "scispacy",
        threshold=0.9,
        umls_subset="rx_sno",
        model="en_core_sci_sm",
        st21pv=False,
        verbose=False,
    )
    elapsed = time.perf_counter() - start
    times["phrase"]["scispacy"]["rx_sno"].append(elapsed)
    start = time.perf_counter()
    CauseNet.is_medical(
        sentence_test_causenet.loc[sentence_test_causenet.dataset == "random_full"].assign(
            cause=sentence_test_causenet.sentence, effect=""
        ),
        "scispacy",
        threshold=0.9,
        umls_subset="full",
        model="en_core_sci_sm",
        st21pv=False,
        verbose=False,
    )
    elapsed = time.perf_counter() - start
    times["sentence"]["scispacy"]["full"].append(elapsed)
    start = time.perf_counter()
    CauseNet.is_medical(
        sentence_test_causenet.loc[sentence_test_causenet.dataset == "random_full"].assign(
            cause=sentence_test_causenet.sentence, effect=""
        ),
        "scispacy",
        threshold=0.9,
        umls_subset="rx_sno",
        model="en_core_sci_sm",
        st21pv=False,
        verbose=False,
    )
    elapsed = time.perf_counter() - start
    times["sentence"]["scispacy"]["rx_sno"].append(elapsed)


def create_index(index, dictionary, index_terms):
    for key, value in dictionary.items():
        index_terms.append(key)
        if isinstance(value, list):
            index.append(tuple(index_terms))
            index_terms = index_terms[:-1]
        else:
            _, index_terms = create_index(index, value, index_terms)
    index_terms = index_terms[:-1]
    return index, index_terms

def grab_values(dictionary):
    values = []
    for value in dictionary.values():
        if isinstance(value, dict):
            values.extend(grab_values(value))
        else:
            values.append(sum(value) / len(value))
    return values

index, _ = create_index([], times, [])
index
values = grab_values(times)
times_df = pd.DataFrame(values, pd.MultiIndex.from_tuples(index, names=["data", "method", "n-gram"]), columns=["time"])
num_samples = pd.Series(
    [
        test_causenet.loc[test_causenet.dataset == "random_full"].shape[0],
        sentence_test_causenet.loc[sentence_test_causenet.dataset == "random_full"].shape[0]
    ],
    index=pd.Index(["phrase", "sentence"], name="data"),
)
times_df["time_per_iter"] = (times_df["time"] / num_samples) * 1000
ratios = times_df["time"].values[:, None] / times_df["time"].values[None, :]
ratio_df = pd.DataFrame(ratios, index=times_df.index, columns=times_df.index)

from IPython.display import display
display(times_df)
display(ratio_df.loc["sentence", "sentence"])
display(ratio_df.loc["phrase", "phrase"])

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Ber

  0%|          | 0/10 [00:00<?, ?it/s]

/opt/conda/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/opt/conda/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/opt/conda/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with sp

/opt/conda/lib/python3.8/site-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
/opt/conda/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/opt/conda/lib/python3.8/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_distances[empty_vectors_boolean_

/opt/conda/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/opt/conda/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/opt/conda/lib/python3.8/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with sp

...

time  time_per_iter
data     method    n-gram                           
sentence termhood  1         1.020202       1.020202
                   2         1.972861       1.972861
                   3         2.820309       2.820309
         bert               47.768734      47.768734
         quickumls full     11.414734      11.414734
                   rx_sno    8.980932       8.980932
         scispacy  full     16.795120      16.795120
                   rx_sno   15.958199      15.958199
         metamap   full    160.638000     160.638000
                   rx_sno  120.277000     120.277000
         ctakes    full      0.000000       0.000000
                   rx_sno  212.711000     212.711000
phrase   termhood  1         0.564155       0.564155
                   2         0.928234       0.928234
                   3         1.265505       1.265505
         bert               60.190908      60.190908
         quickumls full      8.210610       8.210610
                   rx_sno    7.229493       7.229493
         scispacy  full     17.543986      17.543986
                   rx_sno   16.383440      16.383440
         metamap   full     70.639000      70.639000
                   rx_sno   49.636000      49.636000
         ctakes    full      0.000000       0.000000
                   rx_sno  119.684000     119.684000

/tmp/ipykernel_506937/1052718644.py:220: PerformanceWarning: indexing past lexsort depth may impact performance.
  display(ratio_df.loc["sentence", "sentence"])


method              termhood                             bert  quickumls  \
n-gram                     1           2          3                 full   
method    n-gram                                                           
termhood  1         1.000000    0.517118   0.361734  0.021357   0.089376   
          2         1.933794    1.000000   0.699519  0.041300   0.172835   
          3         2.764461    1.429553   1.000000  0.059041   0.247076   
bert               46.822815   24.212928  16.937413  1.000000   4.184831   
quickumls full     11.188698    5.785879   4.047335  0.238958   1.000000   
          rx_sno    8.803091    4.552238   3.184379  0.188009   0.786784   
scispacy  full     16.462542    8.513079   5.955064  0.351592   1.471355   
          rx_sno   15.642194    8.088863   5.658316  0.334072   1.398035   
metamap   full    157.457035   81.423893  56.957592  3.362827  14.072865   
          rx_sno  117.895267   60.965784  42.646748  2.517902  10.536996   
ctakes    full      0.000000    0.000000   0.000000  0.000000   0.000000   
          rx_sno  208.498882  107.818559  75.421173  4.452934  18.634776   

method                        scispacy              metamap           ctakes  \
n-gram               rx_sno       full     rx_sno      full    rx_sno   full   
method    n-gram                                                               
termhood  1        0.113596   0.060744   0.063930  0.006351  0.008482    inf   
          2        0.219672   0.117466   0.123627  0.012281  0.016403    inf   
          3        0.314033   0.167924   0.176731  0.017557  0.023448    inf   
bert               5.318906   2.844203   2.993366  0.297369  0.397156    inf   
quickumls full     1.270997   0.679646   0.715290  0.071059  0.094904    inf   
          rx_sno   1.000000   0.534735   0.562779  0.055908  0.074669    inf   
scispacy  full     1.870086   1.000000   1.052445  0.104553  0.139637    inf   
          rx_sno   1.776898   0.950169   1.000000  0.099343  0.132679    inf   
metamap   full    17.886562   9.564564  10.066173  1.000000  1.335567    inf   
          rx_sno  13.392485   7.161426   7.537003  0.748746  1.000000    inf   
ctakes    full     0.000000   0.000000   0.000000  0.000000  0.000000    NaN   
          rx_sno  23.684735  12.665048  13.329261  1.324164  1.768509    inf   

method                      
n-gram              rx_sno  
method    n-gram            
termhood  1       0.004796  
          2       0.009275  
          3       0.013259  
bert              0.224571  
quickumls full    0.053663  
          rx_sno  0.042221  
scispacy  full    0.078957  
          rx_sno  0.075023  
metamap   full    0.755194  
          rx_sno  0.565448  
ctakes    full    0.000000  
          rx_sno  1.000000

/tmp/ipykernel_506937/1052718644.py:221: PerformanceWarning: indexing past lexsort depth may impact performance.
  display(ratio_df.loc["phrase", "phrase"])


method              termhood                             bert  quickumls  \
n-gram                     1           2          3                 full   
method    n-gram                                                           
termhood  1         1.000000    0.607773   0.445795  0.009373   0.068711   
          2         1.645352    1.000000   0.733489  0.015421   0.113053   
          3         2.243186    1.363347   1.000000  0.021025   0.154130   
bert              106.692117   64.844540  47.562765  1.000000   7.330870   
quickumls full     14.553815    8.845409   6.488011  0.136409   1.000000   
          rx_sno   12.814725    7.788438   5.712735  0.120109   0.880506   
scispacy  full     31.097803   18.900391  13.863231  0.291472   2.136746   
          rx_sno   29.040663   17.650118  12.946169  0.272191   1.995399   
metamap   full    125.212008   76.100422  55.818831  1.173583   8.603381   
          rx_sno   87.982888   53.473585  39.222293  0.824643   6.045349   
ctakes    full      0.000000    0.000000   0.000000  0.000000   0.000000   
          rx_sno  212.147312  128.937314  94.574117  1.988407  14.576750   

method                       scispacy             metamap           ctakes  \
n-gram               rx_sno      full    rx_sno      full    rx_sno   full   
method    n-gram                                                             
termhood  1        0.078035  0.032157  0.034434  0.007986  0.011366    inf   
          2        0.128395  0.052909  0.056657  0.013141  0.018701    inf   
          3        0.175048  0.072133  0.077243  0.017915  0.025496    inf   
bert               8.325744  3.430857  3.673887  0.852092  1.212646    inf   
quickumls full     1.135710  0.468001  0.501153  0.116233  0.165416    inf   
          rx_sno   1.000000  0.412078  0.441268  0.102344  0.145650    inf   
scispacy  full     2.426724  1.000000  1.070837  0.248361  0.353453    inf   
          rx_sno   2.266195  0.933849  1.000000  0.231932  0.330072    inf   
metamap   full     9.770948  4.026394  4.311610  1.000000  1.423140    inf   
          rx_sno   6.865765  2.829232  3.029645  0.702671  1.000000    inf   
ctakes    full     0.000000  0.000000  0.000000  0.000000  0.000000    NaN   
          rx_sno  16.554964  6.821939  7.305182  1.694305  2.411234    inf   

method                      
n-gram              rx_sno  
method    n-gram            
termhood  1       0.004714  
          2       0.007756  
          3       0.010574  
bert              0.502915  
quickumls full    0.068602  
          rx_sno  0.060405  
scispacy  full    0.146586  
          rx_sno  0.136889  
metamap   full    0.590213  
          rx_sno  0.414725  
ctakes    full    0.000000  
          rx_sno  1.000000

In [17]:
display(times_df)
display(ratio_df.loc["sentence", "sentence"])
display(ratio_df.loc["phrase", "phrase"])

time  time_per_iter
data     method    n-gram                           
sentence termhood  1         1.020202       1.020202
                   2         1.972861       1.972861
                   3         2.820309       2.820309
         bert               47.768734      47.768734
         quickumls full     11.414734      11.414734
                   rx_sno    8.980932       8.980932
         scispacy  full     16.795120      16.795120
                   rx_sno   15.958199      15.958199
         metamap   full    160.638000     160.638000
                   rx_sno  120.277000     120.277000
         ctakes    full      0.000000       0.000000
                   rx_sno  212.711000     212.711000
phrase   termhood  1         0.564155       0.564155
                   2         0.928234       0.928234
                   3         1.265505       1.265505
         bert               60.190908      60.190908
         quickumls full      8.210610       8.210610
                   rx_sno    7.229493       7.229493
         scispacy  full     17.543986      17.543986
                   rx_sno   16.383440      16.383440
         metamap   full     70.639000      70.639000
                   rx_sno   49.636000      49.636000
         ctakes    full      0.000000       0.000000
                   rx_sno  119.684000     119.684000

/tmp/ipykernel_506937/2313220437.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  display(ratio_df.loc["sentence", "sentence"])


method              termhood                             bert  quickumls  \
n-gram                     1           2          3                 full   
method    n-gram                                                           
termhood  1         1.000000    0.517118   0.361734  0.021357   0.089376   
          2         1.933794    1.000000   0.699519  0.041300   0.172835   
          3         2.764461    1.429553   1.000000  0.059041   0.247076   
bert               46.822815   24.212928  16.937413  1.000000   4.184831   
quickumls full     11.188698    5.785879   4.047335  0.238958   1.000000   
          rx_sno    8.803091    4.552238   3.184379  0.188009   0.786784   
scispacy  full     16.462542    8.513079   5.955064  0.351592   1.471355   
          rx_sno   15.642194    8.088863   5.658316  0.334072   1.398035   
metamap   full    157.457035   81.423893  56.957592  3.362827  14.072865   
          rx_sno  117.895267   60.965784  42.646748  2.517902  10.536996   
ctakes    full      0.000000    0.000000   0.000000  0.000000   0.000000   
          rx_sno  208.498882  107.818559  75.421173  4.452934  18.634776   

method                        scispacy              metamap           ctakes  \
n-gram               rx_sno       full     rx_sno      full    rx_sno   full   
method    n-gram                                                               
termhood  1        0.113596   0.060744   0.063930  0.006351  0.008482    inf   
          2        0.219672   0.117466   0.123627  0.012281  0.016403    inf   
          3        0.314033   0.167924   0.176731  0.017557  0.023448    inf   
bert               5.318906   2.844203   2.993366  0.297369  0.397156    inf   
quickumls full     1.270997   0.679646   0.715290  0.071059  0.094904    inf   
          rx_sno   1.000000   0.534735   0.562779  0.055908  0.074669    inf   
scispacy  full     1.870086   1.000000   1.052445  0.104553  0.139637    inf   
          rx_sno   1.776898   0.950169   1.000000  0.099343  0.132679    inf   
metamap   full    17.886562   9.564564  10.066173  1.000000  1.335567    inf   
          rx_sno  13.392485   7.161426   7.537003  0.748746  1.000000    inf   
ctakes    full     0.000000   0.000000   0.000000  0.000000  0.000000    NaN   
          rx_sno  23.684735  12.665048  13.329261  1.324164  1.768509    inf   

method                      
n-gram              rx_sno  
method    n-gram            
termhood  1       0.004796  
          2       0.009275  
          3       0.013259  
bert              0.224571  
quickumls full    0.053663  
          rx_sno  0.042221  
scispacy  full    0.078957  
          rx_sno  0.075023  
metamap   full    0.755194  
          rx_sno  0.565448  
ctakes    full    0.000000  
          rx_sno  1.000000

/tmp/ipykernel_506937/2313220437.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  display(ratio_df.loc["phrase", "phrase"])


method              termhood                             bert  quickumls  \
n-gram                     1           2          3                 full   
method    n-gram                                                           
termhood  1         1.000000    0.607773   0.445795  0.009373   0.068711   
          2         1.645352    1.000000   0.733489  0.015421   0.113053   
          3         2.243186    1.363347   1.000000  0.021025   0.154130   
bert              106.692117   64.844540  47.562765  1.000000   7.330870   
quickumls full     14.553815    8.845409   6.488011  0.136409   1.000000   
          rx_sno   12.814725    7.788438   5.712735  0.120109   0.880506   
scispacy  full     31.097803   18.900391  13.863231  0.291472   2.136746   
          rx_sno   29.040663   17.650118  12.946169  0.272191   1.995399   
metamap   full    125.212008   76.100422  55.818831  1.173583   8.603381   
          rx_sno   87.982888   53.473585  39.222293  0.824643   6.045349   
ctakes    full      0.000000    0.000000   0.000000  0.000000   0.000000   
          rx_sno  212.147312  128.937314  94.574117  1.988407  14.576750   

method                       scispacy             metamap           ctakes  \
n-gram               rx_sno      full    rx_sno      full    rx_sno   full   
method    n-gram                                                             
termhood  1        0.078035  0.032157  0.034434  0.007986  0.011366    inf   
          2        0.128395  0.052909  0.056657  0.013141  0.018701    inf   
          3        0.175048  0.072133  0.077243  0.017915  0.025496    inf   
bert               8.325744  3.430857  3.673887  0.852092  1.212646    inf   
quickumls full     1.135710  0.468001  0.501153  0.116233  0.165416    inf   
          rx_sno   1.000000  0.412078  0.441268  0.102344  0.145650    inf   
scispacy  full     2.426724  1.000000  1.070837  0.248361  0.353453    inf   
          rx_sno   2.266195  0.933849  1.000000  0.231932  0.330072    inf   
metamap   full     9.770948  4.026394  4.311610  1.000000  1.423140    inf   
          rx_sno   6.865765  2.829232  3.029645  0.702671  1.000000    inf   
ctakes    full     0.000000  0.000000  0.000000  0.000000  0.000000    NaN   
          rx_sno  16.554964  6.821939  7.305182  1.694305  2.411234    inf   

method                      
n-gram              rx_sno  
method    n-gram            
termhood  1       0.004714  
          2       0.007756  
          3       0.010574  
bert              0.502915  
quickumls full    0.068602  
          rx_sno  0.060405  
scispacy  full    0.146586  
          rx_sno  0.136889  
metamap   full    0.590213  
          rx_sno  0.414725  
ctakes    full    0.000000  
          rx_sno  1.000000

In [10]:
def classify_test_set(test_causenet_predictions, label):
    drop_labels = ["medical_score-cause", "medical_score-effect"]
    pg.set_description(" ".join(label))
    st21pv = "st21pv" in label
    if label[0] == "health_bert":
        _, name, corpus, text_type = label
        model_path = constants.CEPH_PATH + "models/health_bert/"
        model = health_bert.HealthBert.load_from_checkpoint(
            model_path + f"{name}_{corpus}_{text_type}.ckpt"
        )
        medical_score = health_causenet.causenet._health_bert(
            test_causenet_predictions,
            model,
            verbose=False,
        )
    elif label[0] in ("metamap", "ctakes"):
        json_path = os.path.join(
            constants.CEPH_PATH, "tagger_jsons", f"{label[0]}-{label[1]}.jsonl"
        )
        medical_score = CauseNet.is_medical(
            test_causenet_predictions, "tagger", json_path=json_path, st21pv=st21pv
        )
    elif label[0] == "scispacy":
        umls_subset, model, threshold = label[1:4]
        medical_score = CauseNet.is_medical(
            test_causenet_predictions,
            "scispacy",
            umls_subset=umls_subset,
            model=model,
            threshold=float(threshold),
            verbose=False,
            st21pv=st21pv,
        )
    elif label[0] == "quickumls":
        umls_subset, jaccard_threshold = label[1:3]
        jaccard_threshold = float(jaccard_threshold)
        medical_score = CauseNet.is_medical(
            test_causenet_predictions,
            "quickumls",
            jaccard_threshold=jaccard_threshold,
            umls_subset=umls_subset,
            st21pv=st21pv,
            verbose=False,
        )
    elif label[0] in contrastive_scores:
        contrastive_score, corpus, n_gram_size, p_value = label
        n_gram_size = tuple(n_gram_size.strip("()").split(", "))
        n_gram_size = (int(n_gram_size[0]), int(n_gram_size[1]))
        neg = "neg_" in p_value
        if neg:
            p_value = p_value[4:]
        try:
            p_value = int(p_value)
        except:
            p_value = float(p_value)
        if neg:
            p_value = p_value * -1
        medical_score = health_causenet.causenet._contrastive_score(
            test_causenet_predictions,
            medical_termhood[corpus][contrastive_score],
            p=p_value,
            n_gram_size=n_gram_size,
            verbose=False,
        )
    else:
        raise RuntimeError(f"unknown label {label}")
    suffix = "-" + "-".join(label)
    medical_score.index = test_causenet_predictions.index
    medical_score = medical_score.add_suffix(suffix)
    return medical_score

In [11]:
try:
    test_causenet_predictions = pd.read_pickle(
        os.path.join(constants.CEPH_PATH, "test_causenet_predictions.pkl")
    )
except FileNotFoundError:
    test_causenet_predictions = test_causenet.loc[:, ["cause", "effect", "dataset"]]
test_causenet_predictions = pd.read_pickle(
    os.path.join(constants.CEPH_PATH, "test_causenet_predictions.pkl")
)
test_causenet_predictions = test_causenet_predictions.merge(
    test_causenet.drop(
        ["support", "count", "cause_origin", "effect_origin", "evaluation"], axis=1
    ),
    on=["cause", "effect", "dataset"],
    how="outer",
)

jaccard_thresholds = [round(thresh, 2) for thresh in np.arange(0.7, 1.01, 0.1)]
scispacy_thresholds = [round(thresh, 2) for thresh in np.arange(0.6, 0.9, 0.1)]
scispacy_models = ["en_core_sci_sm", "en_core_sci_lg"]
p_values = [-float("inf"), -10, -5, -2, -1, 0, 1, 2, 5, 10, float("inf")]
n_gram_sizes = [(1, 1), (1, 2), (1, 3)]
medical_corpora = [
    "pubmed",
    "textbook",
    "pubmed_central",
    "encyclopedia",
]
umls_subsets = [
    "full",
    "rx_sno",
]
text_types = ["sentence", "noun_phrase"]
contrastive_scores = [
    "term_domain_specificity",
    "contrastive_weight",
    "discriminative_weight",
]
bert_names = ["bert", "scibert", "pubmedbert"]

labels = (
    [
        f"quickumls-{umls_subset}-{jaccard_threshold}-st21pv"
        if st21pv
        else f"quickumls-{umls_subset}-{jaccard_threshold}"
        for st21pv in [True, False]
        for umls_subset in umls_subsets
        for jaccard_threshold in jaccard_thresholds
    ]
    + [
        f"scispacy-{umls_subset}-{model}-{threshold}-st21pv"
        if st21pv
        else f"scispacy-{umls_subset}-{model}-{threshold}"
        for st21pv in [True, False]
        for model in scispacy_models
        for umls_subset in umls_subsets
        for threshold in scispacy_thresholds
    ]
    + [
        f"ctakes-{umls_subset}-st21pv" if st21pv else f"ctakes-{umls_subset}"
        for st21pv in [True, False]
        for umls_subset in umls_subsets
    ]
    + [
        f"metamap-{umls_subset}-st21pv" if st21pv else f"metamap-{umls_subset}"
        for st21pv in [True, False]
        for umls_subset in umls_subsets
    ]
    + [
        f"{contrastive_score}-{medical_corpus}-{n_gram_size}-{p_value}"
        if p_value >= 0
        else f"{contrastive_score}-{medical_corpus}-{n_gram_size}-neg_{-1 * p_value}"
        for contrastive_score in contrastive_scores
        for n_gram_size in n_gram_sizes
        for p_value in p_values
        for medical_corpus in medical_corpora
    ]
    + [
        f"health_bert-{name}-{medical_corpus}-{text_type}"
        for name in bert_names
        for medical_corpus in ["pubmed", "encyclopedia"]
        for text_type in text_types
    ]
)
df_labels = [
    f"medical_score-{relation}-{label}"
    for relation in ["cause", "effect"]
    for label in labels
]
labels = [label.split("-") for label in labels]
new_columns = []
for label in df_labels:
    if label not in test_causenet_predictions:
        new_columns.append(
            pd.Series(np.nan, index=test_causenet_predictions.index, name=label)
        )
test_causenet_predictions = pd.concat([test_causenet_predictions, *new_columns], axis=1)

missing_rows = test_causenet_predictions[df_labels].isna().all(1)
if missing_rows.any():
    print(f"parsing {missing_rows.sum()} missing rows")
    pg = tqdm(labels)
    for label in pg:
        replace_rows = classify_test_set(
            test_causenet_predictions.loc[missing_rows, ["cause", "effect"]], label,
        )
        label = "-".join(label)
        columns = [f"medical_score-cause-{label}", f"medical_score-effect-{label}"]
        replace_rows = replace_rows.loc[:, columns]
        test_causenet_predictions.loc[missing_rows, columns] = replace_rows.values
        test_causenet_predictions.to_pickle(
            os.path.join(constants.CEPH_PATH, "test_causenet_predictions.pkl")
        )

score_column = test_causenet_predictions.columns.str.startswith("medical_score-")
isna = test_causenet_predictions.isna().any(0)
missing_columns = list(test_causenet_predictions.loc[:, score_column & isna])
missing_columns = [
    "-".join(missing_column.split("-")[2:]) for missing_column in missing_columns
]
missing_columns = list(set(missing_columns))
missing_columns = sorted([missing_column.split("-") for missing_column in missing_columns])

if missing_columns:
    print("parsing missing columns")
    for missing_column in missing_columns:
        print(" ".join(missing_column))
    pg = tqdm(missing_columns)
    for label in pg:
        replace_columns = classify_test_set(
            test_causenet_predictions.loc[:, ["cause", "effect"]], label,
        )
        label = "-".join(label)
        columns = [f"medical_score-cause-{label}", f"medical_score-effect-{label}"]
        replace_columns = replace_columns.loc[:, columns]
        test_causenet_predictions.loc[:, columns] = replace_columns.values
        assert not test_causenet_predictions.loc[:, columns].isna().any().any()
        test_causenet_predictions.to_pickle(
            os.path.join(constants.CEPH_PATH, "test_causenet_predictions.pkl")
        )

assert not test_causenet_predictions.loc[:, score_column].isna().any().any()
test_causenet.merge(
    test_causenet_predictions, on=["cause", "effect", "dataset"], how="left"
)

parsing missing columns
contrastive_weight encyclopedia (1, 1) 0
contrastive_weight encyclopedia (1, 1) neg_1
contrastive_weight encyclopedia (1, 1) neg_10
contrastive_weight encyclopedia (1, 1) neg_2
contrastive_weight encyclopedia (1, 1) neg_5
contrastive_weight encyclopedia (1, 1) neg_inf
contrastive_weight encyclopedia (1, 2) 0
contrastive_weight encyclopedia (1, 2) neg_1
contrastive_weight encyclopedia (1, 2) neg_10
contrastive_weight encyclopedia (1, 2) neg_2
contrastive_weight encyclopedia (1, 2) neg_5
contrastive_weight encyclopedia (1, 2) neg_inf
contrastive_weight encyclopedia (1, 3) 0
contrastive_weight encyclopedia (1, 3) neg_1
contrastive_weight encyclopedia (1, 3) neg_10
contrastive_weight encyclopedia (1, 3) neg_2
contrastive_weight encyclopedia (1, 3) neg_5
contrastive_weight encyclopedia (1, 3) neg_inf
contrastive_weight pubmed (1, 1) 0
contrastive_weight pubmed (1, 1) neg_1
contrastive_weight pubmed (1, 1) neg_10
contrastive_weight pubmed (1, 1) neg_2
contrastive_weig

  0%|          | 0/228 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing

,cause,effect,support,count,dataset,cause_origin,effect_origin,evaluation,"medical_score-cause-term_domain_specificity-pubmed-(1, 1)-1","medical_score-effect-term_domain_specificity-pubmed-(1, 1)-1",...,"medical_score-effect-discriminative_weight-pubmed_central-(1, 3)-neg_2","medical_score-effect-discriminative_weight-encyclopedia-(1, 3)-neg_2","medical_score-effect-discriminative_weight-pubmed-(1, 3)-neg_1","medical_score-effect-discriminative_weight-textbook-(1, 3)-neg_1","medical_score-effect-discriminative_weight-pubmed_central-(1, 3)-neg_1","medical_score-effect-discriminative_weight-encyclopedia-(1, 3)-neg_1","medical_score-effect-discriminative_weight-pubmed-(1, 3)-0","medical_score-effect-discriminative_weight-textbook-(1, 3)-0","medical_score-effect-discriminative_weight-pubmed_central-(1, 3)-0","medical_score-effect-discriminative_weight-encyclopedia-(1, 3)-0"
0,accident,death,38.0,1458.0,support,NaN,NaN,0,0.271897,0.580596,...,57.204029,18.803064,65.054640,21.417447,57.204029,18.803064,65.054640,21.417447,57.204029,18.803064
1,pneumonia,death,37.0,1185.0,support,NaN,NaN,1,2.250561,0.580596,...,57.204029,18.803064,65.054640,21.417447,57.204029,18.803064,65.054640,21.417447,57.204029,18.803064
2,disease,death,37.0,1344.0,support,NaN,NaN,1,3.030815,0.580596,...,57.204029,18.803064,65.054640,21.417447,57.204029,18.803064,65.054640,21.417447,57.204029,18.803064
3,illness,death,36.0,571.0,support,NaN,NaN,1,1.511752,0.580596,...,57.204029,18.803064,65.054640,21.417447,57.204029,18.803064,65.054640,21.417447,57.204029,18.803064
4,heart attack,death,36.0,1005.0,support,NaN,NaN,1,0.752565,0.580596,...,57.204029,18.803064,65.054640,21.417447,57.204029,18.803064,65.054640,21.417447,57.204029,18.803064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14311,initial oxidation at carbon 5,thymine glycol,NaN,NaN,practitioner_unsure,NaN,NaN,1,1.309622,2.876154,...,276.292460,127.148997,417.882586,20.809708,286.801086,128.436658,431.847518,21.764573,298.182251,129.764249
14312,thyroid hormone deficits,language,NaN,NaN,practitioner_sure,NaN,NaN,0,3.480315,0.250982,...,41.967722,4.199330,27.781071,14.799191,41.967722,4.199330,27.781071,14.799191,41.967722,4.199330
14313,neuropeptide profile,decreased food intake,NaN,NaN,practitioner_sure,NaN,NaN,0,2.542437,2.515899,...,238.498888,94.748032,284.867129,58.498232,268.957121,102.036191,338.810479,65.615679,297.029307,110.575203
14314,disorder,white heads,NaN,NaN,practitioner_sure,NaN,NaN,0,2.494475,0.247209,...,32.916396,3.574758,16.025289,6.810649,33.184736,4.423016,18.603061,6.827452,33.459747,6.457744


In [12]:
try:
    sentence_test_causenet_predictions = pd.read_pickle(
        os.path.join(constants.CEPH_PATH, "sentence_test_causenet_predictions.pkl")
    )
except FileNotFoundError:
    sentence_test_causenet_predictions = sentence_test_causenet.loc[
        :, ["cause", "effect", "dataset", "sentence"]
    ]

sentence_test_causenet_predictions = sentence_test_causenet_predictions.merge(
    sentence_test_causenet.drop(
        [
            "support",
            "count",
            "cause_origin",
            "effect_origin",
            "evaluation",
            "manual_evaluation",
        ],
        axis=1,
    ),
    on=["cause", "effect", "sentence", "dataset"],
    how="outer",
)

jaccard_thresholds = [round(thresh, 2) for thresh in np.arange(0.7, 1.01, 0.1)]
scispacy_thresholds = [round(thresh, 2) for thresh in np.arange(0.6, 0.9, 0.1)]
scispacy_models = ["en_core_sci_sm", "en_core_sci_lg"]
p_values = [-float("inf"), -10, -5, -2, -1, 0, 1, 2, 5, 10, float("inf")]
n_gram_sizes = [(1, 1), (1, 2), (1, 3)]
medical_corpora = [
    "pubmed",
    "textbook",
    "pubmed_central",
    "encyclopedia",
]
umls_subsets = [
    "full",
    "rx_sno",
]
text_types = ["sentence", "noun_phrase"]
contrastive_scores = [
    "term_domain_specificity",
    "contrastive_weight",
    "discriminative_weight",
]
bert_names = ["bert", "scibert", "pubmedbert"]

labels = (
    [
        f"quickumls-{umls_subset}-{jaccard_threshold}-st21pv"
        if st21pv
        else f"quickumls-{umls_subset}-{jaccard_threshold}"
        for st21pv in [True, False]
        for umls_subset in umls_subsets
        for jaccard_threshold in jaccard_thresholds
    ]
    + [
        f"scispacy-{umls_subset}-{model}-{threshold}-st21pv"
        if st21pv
        else f"scispacy-{umls_subset}-{model}-{threshold}"
        for st21pv in [True, False]
        for model in scispacy_models
        for umls_subset in umls_subsets
        for threshold in scispacy_thresholds
    ]
    + [
        f"ctakes-{umls_subset}-st21pv" if st21pv else f"ctakes-{umls_subset}"
        for st21pv in [True, False]
        for umls_subset in umls_subsets
    ]
    + [
        f"metamap-{umls_subset}-st21pv" if st21pv else f"metamap-{umls_subset}"
        for st21pv in [True, False]
        for umls_subset in umls_subsets
    ]
    + [
        f"{contrastive_score}-{medical_corpus}-{n_gram_size}-{p_value}"
        if p_value >= 0
        else f"{contrastive_score}-{medical_corpus}-{n_gram_size}-neg_{-1 * p_value}"
        for contrastive_score in contrastive_scores
        for n_gram_size in n_gram_sizes
        for p_value in p_values
        for medical_corpus in medical_corpora
    ]
    + [
        f"health_bert-{name}-{medical_corpus}-{text_type}"
        for name in bert_names
        for medical_corpus in ["pubmed", "encyclopedia"]
        for text_type in text_types
    ]
)
df_labels = [
    f"medical_score-{relation}-{label}"
    for relation in ["cause", "effect"]
    for label in labels
]
labels = [label.split("-") for label in labels]
new_columns = []
for label in df_labels:
    if label not in sentence_test_causenet_predictions:
        new_columns.append(
            pd.Series(
                np.nan, index=sentence_test_causenet_predictions.index, name=label
            )
        )
sentence_test_causenet_predictions = pd.concat(
    [sentence_test_causenet_predictions, *new_columns], axis=1
)

prediction_sentence_test_causenet = sentence_test_causenet_predictions.copy()
prediction_sentence_test_causenet.cause = prediction_sentence_test_causenet.sentence
prediction_sentence_test_causenet.effect = ""
prediction_sentence_test_causenet = prediction_sentence_test_causenet.drop(
    ["sentence",], axis=1,
)

missing_rows = sentence_test_causenet_predictions[df_labels].isna().all(1)
if missing_rows.any():
    print(f"parsing {missing_rows.sum()} missing rows")
    pg = tqdm(labels)
    for label in pg:
        replace_rows = classify_test_set(
            prediction_sentence_test_causenet.loc[missing_rows, ["cause", "effect"]],
            label,
        )
        label = "-".join(label)
        columns = [f"medical_score-cause-{label}", f"medical_score-effect-{label}"]
        replace_rows = replace_rows.loc[:, columns]
        sentence_test_causenet_predictions.loc[
            missing_rows, columns
        ] = replace_rows.values
        
        effect_columns = sentence_test_causenet_predictions.columns[
            sentence_test_causenet_predictions.columns.str.startswith("medical_score-effect")
        ]
        sentence_test_causenet_predictions.loc[:, effect_columns] = 0
        sentence_test_causenet_predictions.to_pickle(
            os.path.join(constants.CEPH_PATH, "sentence_test_causenet_predictions.pkl")
        )
         
score_column = sentence_test_causenet_predictions.columns.str.startswith(
    "medical_score-"
)
isna = sentence_test_causenet_predictions.isna().any(0)
missing_columns = list(sentence_test_causenet_predictions.loc[:, score_column & isna])
missing_columns = ["-".join(missing_column.split("-")[2:]) for missing_column in missing_columns]
missing_columns = list(set(missing_columns))
missing_columns = sorted([missing_column.split("-") for missing_column in missing_columns])

if missing_columns:
    print("parsing missing columns")
    for missing_column in missing_columns:
        print(" ".join(missing_column))
    pg = tqdm(missing_columns)
    for label in pg:
        replace_columns = classify_test_set(
            prediction_sentence_test_causenet.loc[:, ["cause", "effect"]], label,
        )
        label = "-".join(label)
        columns = [f"medical_score-cause-{label}", f"medical_score-effect-{label}"]
        replace_columns = replace_columns.loc[:, columns]
        sentence_test_causenet_predictions.loc[:, columns] = replace_columns.values
        assert not sentence_test_causenet_predictions.loc[:, columns].isna().any().any()

        effect_columns = sentence_test_causenet_predictions.columns[
            sentence_test_causenet_predictions.columns.str.startswith("medical_score-effect")
        ]
        sentence_test_causenet_predictions.loc[:, effect_columns] = 0
        sentence_test_causenet_predictions.to_pickle(
            os.path.join(constants.CEPH_PATH, "sentence_test_causenet_predictions.pkl")
        )
        
assert not sentence_test_causenet_predictions.loc[:, score_column].isna().any().any()

sentence_test_causenet.merge(
    sentence_test_causenet_predictions,
    on=["cause", "effect", "sentence", "dataset"],
    how="left",
)

parsing missing columns
contrastive_weight encyclopedia (1, 1) 0
contrastive_weight encyclopedia (1, 1) neg_1
contrastive_weight encyclopedia (1, 1) neg_10
contrastive_weight encyclopedia (1, 1) neg_2
contrastive_weight encyclopedia (1, 1) neg_5
contrastive_weight encyclopedia (1, 1) neg_inf
contrastive_weight encyclopedia (1, 2) 0
contrastive_weight encyclopedia (1, 2) neg_1
contrastive_weight encyclopedia (1, 2) neg_10
contrastive_weight encyclopedia (1, 2) neg_2
contrastive_weight encyclopedia (1, 2) neg_5
contrastive_weight encyclopedia (1, 2) neg_inf
contrastive_weight encyclopedia (1, 3) 0
contrastive_weight encyclopedia (1, 3) neg_1
contrastive_weight encyclopedia (1, 3) neg_10
contrastive_weight encyclopedia (1, 3) neg_2
contrastive_weight encyclopedia (1, 3) neg_5
contrastive_weight encyclopedia (1, 3) neg_inf
contrastive_weight pubmed (1, 1) 0
contrastive_weight pubmed (1, 1) neg_1
contrastive_weight pubmed (1, 1) neg_10
contrastive_weight pubmed (1, 1) neg_2
contrastive_weig

  0%|          | 0/228 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing

,cause,effect,support,count,dataset,cause_origin,effect_origin,evaluation,sentence,manual_evaluation,...,medical_score-effect-health_bert-bert-encyclopedia-sentence,medical_score-effect-health_bert-bert-encyclopedia-noun_phrase,medical_score-effect-health_bert-scibert-pubmed-sentence,medical_score-effect-health_bert-scibert-pubmed-noun_phrase,medical_score-effect-health_bert-scibert-encyclopedia-sentence,medical_score-effect-health_bert-scibert-encyclopedia-noun_phrase,medical_score-effect-health_bert-pubmedbert-pubmed-sentence,medical_score-effect-health_bert-pubmedbert-pubmed-noun_phrase,medical_score-effect-health_bert-pubmedbert-encyclopedia-sentence,medical_score-effect-health_bert-pubmedbert-encyclopedia-noun_phrase
0,accident,death,38.0,1458.0,support,NaN,NaN,0,"For example, the book also described how to di...",NaN,...,0,0,0,0,0,0,0,0,0,0
1,pneumonia,death,37.0,1185.0,support,NaN,NaN,1,Pneumonia was a serious cause of death in the ...,NaN,...,0,0,0,0,0,0,0,0,0,0
2,disease,death,37.0,1344.0,support,NaN,NaN,1,This is a stage in some diseases before the sy...,NaN,...,0,0,0,0,0,0,0,0,0,0
3,illness,death,36.0,571.0,support,NaN,NaN,1,A primary cause of death was illness due to ha...,NaN,...,0,0,0,0,0,0,0,0,0,0
4,heart attack,death,36.0,1005.0,support,NaN,NaN,1,His death was announced to Lok Sabha at 14:00 ...,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,termination of the company 's pexelizumab prog...,non-recurrence of costs,1.0,3.0,random_full,NaN,NaN,0,The decrease in R&D expenses in 2007 reflected...,0.0,...,0,0,0,0,0,0,0,0,0,0
2996,medical neglect of a patient at another nursin...,abuse,1.0,2.0,random_full,NaN,NaN,0,That dispute arose when HRS found indications ...,0.0,...,0,0,0,0,0,0,0,0,0,0
2997,disorder,shoulder contractions,1.0,8.0,random_full,NaN,NaN,1,This is a disorder that causes severe neck and...,1.0,...,0,0,0,0,0,0,0,0,0,0
2998,car accidents,rib fracture in a child,1.0,1.0,random_full,NaN,NaN,1,A rib fracture in a child is mainly caused by ...,1.0,...,0,0,0,0,0,0,0,0,0,0
